In [66]:
from matplotlib import pyplot as plt
import numpy as np
import json
import os
from tqdm import tqdm
import pandas as pd
import pickle
import re
import random

In [67]:
root_dir = 'KorWikiTQ'
data_files = os.listdir(root_dir)

In [68]:
for path in data_files:
    with open(os.path.join(root_dir, path), 'rt', encoding='UTF8') as f:
        data = json.load(f)

    errow_rows = []
    possible_tqa = []
    print(f"> Processing {len(data['data'])} tables")
    for i, tqa in enumerate(tqdm(data['data'])):
        header = tqa['TBL'][0]
        if '' in header:
            errow_rows.append(i)
            continue
        num_cols = len(header)
        error_in_table = False
        for row in tqa['TBL']:
            if len(row) != num_cols:
                errow_rows.append((f'data_idx:{i}', row))
                error_in_table = True
        if not error_in_table:
            cells = np.array(tqa['TBL'])[1:, 2:]
            try:
                cells = cells.astype(np.float)
                possible_tqa.append(tqa)
            except:
                continue
    # possible_tqa = [dict(t) for t in set(tuple(sorted(d.items())) for d in possible_tqa)]
    print(f"> Found {len(possible_tqa)} tables with numeric values ({len(possible_tqa)/len(data['data'])*100:.1f}%)")

    colors = ['black', 'dimgray', 'dimgrey', 'gray', 'grey', 'darkgray', 'darkgrey', 'silver', 'lightgray', 'lightgrey', 'gainsboro', 'whitesmoke', 'white', 'snow', 'rosybrown', 'lightcoral', 'indianred', 'brown', 'firebrick', 'maroon', 'darkred', 'red', 'mistyrose', 'salmon', 'tomato', 'darksalmon', 'coral', 'orangered', 'lightsalmon', 'sienna', 'seashell', 'chocolate', 'saddlebrown', 'sandybrown', 'peachpuff', 'peru', 'linen', 'bisque', 'darkorange', 'burlywood', 'antiquewhite', 'tan', 'navajowhite', 'blanchedalmond', 'papayawhip', 'moccasin', 'orange', 'wheat', 'oldlace', 'floralwhite', 'darkgoldenrod', 'goldenrod', 'cornsilk', 'gold', 'lemonchiffon', 'khaki', 'palegoldenrod', 'darkkhaki', 'ivory', 'beige', 'lightyellow', 'lightgoldenrodyellow', 'olive', 'yellow', 'olivedrab', 'yellowgreen', 'darkolivegreen', 'greenyellow', 'chartreuse', 'lawngreen', 'honeydew', 'darkseagreen', 'palegreen', 'lightgreen', 'forestgreen', 'limegreen', 'darkgreen', 'green', 'lime', 'seagreen', 'mediumseagreen', 'springgreen', 'mintcream', 'mediumspringgreen', 'mediumaquamarine', 'aquamarine', 'turquoise', 'lightseagreen', 'mediumturquoise', 'azure', 'lightcyan', 'paleturquoise', 'darkslategray', 'darkslategrey', 'teal', 'darkcyan', 'aqua', 'cyan', 'darkturquoise', 'cadetblue', 'powderblue', 'lightblue', 'deepskyblue', 'skyblue', 'lightskyblue', 'steelblue', 'aliceblue', 'dodgerblue', 'lightslategray', 'lightslategrey', 'slategray', 'slategrey', 'lightsteelblue', 'cornflowerblue', 'royalblue', 'ghostwhite', 'lavender', 'midnightblue', 'navy', 'darkblue', 'mediumblue', 'blue', 'slateblue', 'darkslateblue', 'mediumslateblue', 'mediumpurple', 'rebeccapurple', 'blueviolet', 'indigo', 'darkorchid', 'darkviolet', 'mediumorchid', 'thistle', 'plum', 'violet', 'purple', 'darkmagenta', 'fuchsia', 'magenta', 'orchid', 'mediumvioletred', 'deeppink', 'hotpink', 'lavenderblush', 'palevioletred', 'crimson', 'pink', 'lightpink']



 52%|█████▏    | 6111/11771 [00:00<00:00, 30200.89it/s]

> Processing 11771 tables


100%|██████████| 11771/11771 [00:00<00:00, 29213.91it/s]


> Found 106 tables with numeric values (0.9%)


 10%|▉         | 5640/58221 [00:00<00:01, 27641.13it/s]

> Processing 58221 tables


100%|██████████| 58221/58221 [00:02<00:00, 29072.01it/s]

> Found 472 tables with numeric values (0.8%)


In [69]:
import matplotlib
import random 
from PIL import Image, ImageDraw

def visual_bbox(bbox_list, plt_size, fig_name, bbfig_name):
    #-------------------------Example-------------------------#
    width, height = plt_size
    color_names = list(matplotlib.colors.cnames.keys())
    random.shuffle(color_names)
    # color_names = list(mcolors.CSS4_COLORS)
    img = Image.open(fig_name).convert('RGB')
    draw = ImageDraw.Draw(img)
    box_color = []
    for i, l in enumerate(bbox_list):
        bbox_cls = l[0]
        if bbox_cls not in box_color:
            box_color.append(bbox_cls)
            # print(bbox_cls, color_names[len(box_color)])
        bb = l[1]
        draw.rectangle((bb[0], height-bb[1], bb[2], height-bb[3]), outline=color_names[len(box_color)], width = 5)
    img.save(bbfig_name)
    # img.show()
    # img.save(f'./line_bbox/linebbox_{index}.png')
    return 

In [70]:
plt.ioff()
final_result = []

img_idx = 0
for anno_id in range(1):
    for idx, tqa in enumerate(possible_tqa):
        if idx > 10 : break
        # print(tqa['QAS']['qid'])
        #tqa = possible_tqa[-14]
        table = tqa['TBL']

        dummy_list = ['조', '순위', '번호', '시드', 'rank']
        new_table = []
        if table[0][0] in dummy_list:
            new_columns = table[0][1:] + [table[0][0]]
            df_dropped = pd.DataFrame(table[1:],columns=table[0])[new_columns]
            # inter_table.append(list(df_dropped.columns))
            new_table.append(new_columns)
            for i, row in df_dropped.iterrows():
                new_table.append(list(row.values))
        else:
            new_table = table.copy()
        table = new_table

        candidates = re.sub(r'[^0-9 ]', '', tqa['QAS']['question'])
        candidates = candidates.split() + [tqa['QAS']['answer']]

        flag = 0
        correct_coord = []
        for i in range(len(table[0])):
            if table[0][i] in tqa['QAS']['question']:
                correct_coord.append([0,i])
        for i in range(1,len(table)):
            for cand in candidates:
                try:
                    j = table[i].index(cand)
                    correct_coord.append([i,j])
                    flag=1
                except:
                    continue
        if flag==1: # table 축소
            table_temp = table.copy()
            selected_table = []
            full_idx = list(np.arange(len(table_temp)))
            num_row = np.random.randint(2,4)
            row_idx = random.sample(full_idx[1:],num_row)
            correct_row = np.unique(np.array(np.array(correct_coord)[:,0].tolist()+row_idx)).tolist()[::-1]
            for i in correct_row:
                if i == 0:
                    pass
                else:
                    selected_table.append(table_temp[i].copy())
            
            selected_table.sort()
            selected_table.insert(0, table_temp[0])
            
            full_idx = list(np.arange(len(selected_table[0])))
            num_col = np.random.randint(2,4)
            col_idx = random.sample(full_idx[1:],num_col)
            col_idx.append(0)
            correct_col = np.array(correct_coord)[:,1].tolist()
            col_idx.extend(correct_col)
            col_idx.sort()
            col_idx = np.unique(np.array(col_idx)).tolist()
            selected_table = pd.DataFrame(selected_table)[col_idx].values.tolist()
            table=selected_table

        dpi = 300
        plt.rcParams['figure.dpi'] = dpi
        plt.rcParams['font.size'] = 8.
        plt.rcParams["font.family"] = 'NanumGothicCoding'
        plt.rcParams['xtick.labelsize'] = 8.
        plt.rcParams['ytick.labelsize'] = 15.
        plt.rcParams['axes.labelsize'] = 15.
        plt.rcParams["figure.figsize"] = (15,8)


        fig, ax = plt.subplots()
        # TBL = np.array(tqa['TBL'])
        TBL = np.array(table)

        header = TBL[0][1:]
        x = TBL[1:, 0]
        x_label = TBL[0][0]

        y_list = []
        y_label = [l for l in TBL[0][1:]]
        y_label =  ' '.join(y_label)
        try:
            for y in TBL[1:, 1:].T:
                y_list.append(y.astype(np.float))
        except:
            continue

        plt.xlabel(x_label, labelpad=7)
        plt.ylabel(y_label, labelpad=7)

        selected_colors = np.random.choice(colors, len(y_list), replace=False)
        label2color = {label:selected_colors[i] for i, label in enumerate(header)}

        handles = [plt.Rectangle((0,0), 0, 0, color=label2color[label]) for i, label in enumerate(header)]
        plt.legend(handles, header,fontsize=10)

        x_axis = np.arange(len(x))*2

        num_y = len(y_list)

        num_bars_per_tick = len(y_list)
        wid = 0.2
        diff_w = [wid * (i - (num_bars_per_tick-1)/2) for i in range(num_bars_per_tick)]
        bar_list = []
        val_list = []
        if TBL[1:, 1:].astype(np.float).min() < 0:
            ylim_min = TBL[1:, 1:].astype(np.float).min() * 1.7
        else:
            ylim_min = 0
        plt.ylim(ylim_min, TBL[1:, 1:].astype(np.float).max() * 1.2)
        # print(img_idx, ylim_min, TBL[1:, 1:].astype(np.float).max() * 1.4)

        for i, (yy, label) in enumerate(zip(y_list, header)):
            a = []
            for v in yy:
                if  str(v)[-2:]==".0": 
                    a.append(int(re.sub(".0$", "", str(v))))
                else :
                    a.append(round(v,2))
            y = a
            x_axis_temp = x_axis + diff_w[i]
            temp_bar = plt.bar(x_axis + diff_w[i], y, wid, color=label2color[label])
            bar_list.append(temp_bar)
            for i, val in enumerate(y):
                if val >= 0:
                    temp_val = plt.text(x_axis_temp[i], val+0.2, val,
                                fontsize = 12, 
                                horizontalalignment='center',  # horizontalalignment (left, center, right)
                                verticalalignment='bottom')    # verticalalignment (top, center, bottom)
                else:
                    temp_val = plt.text(x_axis_temp[i], val-0.2, val,
                                fontsize = 12, 
                                horizontalalignment='center',  # horizontalalignment (left, center, right)
                                verticalalignment='top')    # verticalalignment (top, center, bottom)
                val_list.append(temp_val)
        plt.xticks(x_axis, x, fontsize=14)
        flatten = [item for sublist in y_list for item in sublist]
        min_y, max_y = min(flatten), max(flatten)
        y_tick = np.arange(min_y, max_y+(max_y-min_y)/5, (max_y-min_y)/5)
        y_tick = [round(item,2) for item in y_tick]
        plt.yticks(y_tick, y_tick, fontsize=14)
        # fig.autofmt_xdate()
        try:
            plt.tight_layout()
        except:
            continue
        plt.savefig('data/bar/bar_'+str(img_idx)+'.png', dpi=dpi, facecolor='white', transparent=False)

        val_list_for_bbox = []
        val_sublist = []
        cnt = 0
        for i, val in enumerate(val_list):
            x = i//len(bar_list[0])
            if x > cnt:
                val_list_for_bbox.append(val_sublist)
                val_sublist = []
                cnt += 1
            val_sublist.append(val)
        val_list_for_bbox.append(val_sublist)

        bboxes = []
        (x_min, y_min), (x_max, y_max) = ax.bbox.get_points()
        #x label
        box = np.array(ax.get_xaxis().get_label().get_window_extent(None).get_points())
        bboxes.append(("x_label", [box[0][0], box[0][1], box[1][0], box[1][1]], str(x_label)))

        #y label
        box = np.array(ax.get_yaxis().get_label().get_window_extent(None).get_points())
        bboxes.append(("y_label", [box[0][0], box[0][1], box[1][0], box[1][1]], str(y_label)))

        ## bbox for ticks
        xtick_label_idx = 0
        for label in ax.get_xticklabels():
            bbox = label.get_window_extent().get_points()
        #     if bbox[0][0] < x_min or bbox[1][0] > x_max:
        #         continue
            (x_min_bbox, y_min_bbox), (x_max_bbox, y_max_bbox) = bbox
            label = label.get_text().split('\n')
            height = y_max_bbox - y_min_bbox
            num_lines = len(label)
            sub_height = height/num_lines
            for i, tb in enumerate(label):
                new_bbox = [x_min_bbox, y_min_bbox + sub_height*i, x_max_bbox, y_min_bbox + sub_height*(i+1)]
                bboxes.append((f'x_tick', new_bbox, tb))
                # bboxes.append((f'xtick_{xtick_label_idx}', bbox))
                xtick_label_idx += 1

        ytick_label_idx = 0
        for label in ax.get_yticklabels():
            bbox = label.get_window_extent().get_points()
            # if bbox[0][1] < y_min or bbox[1][1] > y_max:
            #     continue
            (x_min_bbox, y_min_bbox), (x_max_bbox, y_max_bbox) = bbox
            label = label.get_text().split('\n')
            height = y_max_bbox - y_min_bbox
            num_lines = len(label)
            sub_height = height/num_lines
            for i, tb in enumerate(label):
                new_bbox = [x_min_bbox, y_min_bbox + sub_height*i, x_max_bbox, y_min_bbox + sub_height*(i+1)]
                bboxes.append((f'y_tick', new_bbox, tb))
                # bboxes.append((f'xtick_{xtick_label_idx}', bbox))
                ytick_label_idx += 1
                
        # Bar
        for j in range(len(bar_list[0])):
            for i in range(len(bar_list)):
                bbox = bar_list[i][j].get_window_extent().get_points()
                (x_min, y_min), (x_max, y_max) = bbox
                new_bbox = [x_min, y_min, x_max, y_max]
                bboxes.append((f'bar', new_bbox, None))
                
        # Value
        for j in range(len(val_list_for_bbox[0])):
            for i in range(len(val_list_for_bbox)):
                bbox = val_list_for_bbox[i][j].get_window_extent().get_points()
                (x_min, y_min), (x_max, y_max) = bbox
                new_bbox = [x_min, y_min, x_max, y_max]
                bboxes.append((f'val', new_bbox, val_list_for_bbox[i][j].get_text()))
                
        # legend
        for i, patch in enumerate(ax.get_legend().get_patches()):
            bbox = patch.get_window_extent().get_points()
            (x_min, y_min), (x_max, y_max) = bbox
            new_bbox = [x_min, y_min, x_max, y_max]
            bboxes.append((f'v_legend', new_bbox, None))


        for i, label in enumerate(ax.get_legend().get_texts()):
            bbox = label.get_window_extent().get_points()
            (x_min, y_min), (x_max, y_max) = bbox
            new_bbox = [x_min, y_min, x_max, y_max]
            bboxes.append((f't_legend', new_bbox, label.get_text()))

        bbox = fig.get_window_extent(None).transformed(fig.dpi_scale_trans.inverted())
        plt_size = bbox.width*fig.dpi, bbox.height*fig.dpi
        visual_bbox(bboxes, plt_size, 'data/bar/bar_'+str(img_idx)+'.png', 'data/bar_bbox/bar_bbox_'+str(img_idx)+'.png')

        final_elem = {
            'qid':tqa['QAS']['qid'],
            'question':tqa['QAS']['question'],
            'answer':tqa['QAS']['answer'],
            'image':'bar_'+ str(img_idx) + '.png' ,
            'bboxes':bboxes
        }
        img_idx += 1
        final_result.append(final_elem)
        fig.clf()
        plt.close(fig)

# with open('bar_annotation.pkl',"wb") as f:
#     pickle.dump(final_result, f)

# with open('bar_annotation.pkl', 'rb') as handle:
#     print(pickle.load(handle))

In [71]:
plt.ioff()
final_result = []

img_idx = 0
for anno_id in range(1):
    for idx, tqa in enumerate(possible_tqa):
        if idx > 10 : break
        # print(tqa['QAS']['qid'])
        #tqa = possible_tqa[-14]
        table = tqa['TBL']

        dummy_list = ['조', '순위', '번호', '시드', 'rank']
        new_table = []
        if table[0][0] in dummy_list:
            new_columns = table[0][1:] + [table[0][0]]
            df_dropped = pd.DataFrame(table[1:],columns=table[0])[new_columns]
            # inter_table.append(list(df_dropped.columns))
            new_table.append(new_columns)
            for i, row in df_dropped.iterrows():
                new_table.append(list(row.values))
        else:
            new_table = table.copy()
        table = new_table

        candidates = re.sub(r'[^0-9 ]', '', tqa['QAS']['question'])
        candidates = candidates.split() + [tqa['QAS']['answer']]

        flag = 0
        correct_coord = []
        for i in range(len(table[0])):
            if table[0][i] in tqa['QAS']['question']:
                correct_coord.append([0,i])
        for i in range(1,len(table)):
            for cand in candidates:
                try:
                    j = table[i].index(cand)
                    correct_coord.append([i,j])
                    flag=1
                except:
                    continue
        if flag==1: # table 축소
            table_temp = table.copy()
            selected_table = []
            full_idx = list(np.arange(len(table_temp)))
            num_row = np.random.randint(2,4)
            row_idx = random.sample(full_idx[1:],num_row)
            correct_row = np.unique(np.array(np.array(correct_coord)[:,0].tolist()+row_idx)).tolist()[::-1]
            for i in correct_row:
                if i == 0:
                    pass
                else:
                    selected_table.append(table_temp[i].copy())
            
            selected_table.sort()
            selected_table.insert(0, table_temp[0])
            
            full_idx = list(np.arange(len(selected_table[0])))
            num_col = np.random.randint(2,4)
            col_idx = random.sample(full_idx[1:],num_col)
            col_idx.append(0)
            correct_col = np.array(correct_coord)[:,1].tolist()
            col_idx.extend(correct_col)
            col_idx.sort()
            col_idx = np.unique(np.array(col_idx)).tolist()
            selected_table = pd.DataFrame(selected_table)[col_idx].values.tolist()
            table=selected_table

        dpi = 300
        plt.rcParams['figure.dpi'] = dpi
        plt.rcParams['font.size'] = 8.
        plt.rcParams["font.family"] = 'NanumGothicCoding'
        plt.rcParams['xtick.labelsize'] = 15.
        plt.rcParams['ytick.labelsize'] = 8.
        plt.rcParams['axes.labelsize'] = 15.
        plt.rcParams["figure.figsize"] = (15,8)


        fig, ax = plt.subplots()
        # TBL = np.array(tqa['TBL'])
        TBL = np.array(table)

        header = TBL[0][1:]
        x = TBL[1:, 0]
        x_label = TBL[0][0]

        y_list = []
        y_label = [l for l in TBL[0][1:]]
        y_label =  ' '.join(y_label)
        try:
            for y in TBL[1:, 1:].T:
                y_list.append(y.astype(np.float))
        except:
            continue

        plt.xlabel(y_label, labelpad=7)
        plt.ylabel(x_label, labelpad=7)

        selected_colors = np.random.choice(colors, len(y_list), replace=False)
        label2color = {label:selected_colors[i] for i, label in enumerate(header)}

        handles = [plt.Rectangle((0,0), 0, 0, color=label2color[label]) for i, label in enumerate(header)]
        plt.legend(handles, header,fontsize=10)

        x_axis = np.arange(len(x))*2

        num_y = len(y_list)

        num_bars_per_tick = len(y_list)
        wid = 0.2
        diff_w = [wid * (i - (num_bars_per_tick-1)/2) for i in range(num_bars_per_tick)]
        bar_list = []
        val_list = []
        if TBL[1:, 1:].astype(np.float).min() < 0:
            ylim_min = TBL[1:, 1:].astype(np.float).min() * 1.7
        else:
            ylim_min = 0
        plt.xlim(ylim_min, TBL[1:, 1:].astype(np.float).max() * 1.2)
        # print(img_idx, ylim_min, TBL[1:, 1:].astype(np.float).max() * 1.4)

        fig_width, fig_height = plt.gcf().get_size_inches()

        for i, (yy, label) in enumerate(zip(y_list, header)):
            a = []
            for v in yy:
                if  str(v)[-2:]==".0": 
                    a.append(int(re.sub(".0$", "", str(v))))
                else :
                    a.append(round(v,2))
            y = a
            # print(x_axis[i], x_axis_temp[i])
            x_axis_temp = x_axis + diff_w[i]
            temp_bar = plt.barh(x_axis + diff_w[i], y, wid, color=label2color[label])
            bar_list.append(temp_bar)
            for bar, xval, val in zip(ax.patches, x_axis_temp, y):
                
                if val >=0:
                    temp_val = ax.text(val+bar.get_height()+fig_width/150, xval, val, ha = 'center', va = 'center', fontsize = 12)
                else:
                    temp_val = ax.text(val-bar.get_height()-fig_width/150, xval, val, ha = 'center', va = 'center', fontsize = 12)
                val_list.append(temp_val)
        plt.yticks(x_axis, x, fontsize=14)
        flatten = [item for sublist in y_list for item in sublist]
        min_y, max_y = min(flatten), max(flatten)
        y_tick = np.arange(min_y, max_y+(max_y-min_y)/5, (max_y-min_y)/5)
        y_tick = [round(item,2) for item in y_tick]
        plt.xticks(y_tick, y_tick, fontsize=14)
        # fig.autofmt_xdate()
        try:
            plt.tight_layout()
        except:
            continue
        plt.savefig('data/bar/barh_'+str(img_idx)+'.png', dpi=dpi, facecolor='white', transparent=False)

        val_list_for_bbox = []
        val_sublist = []
        cnt = 0
        for i, val in enumerate(val_list):
            x = i//len(bar_list[0])
            if x > cnt:
                val_list_for_bbox.append(val_sublist)
                val_sublist = []
                cnt += 1
            val_sublist.append(val)
        val_list_for_bbox.append(val_sublist)

        bboxes = []
        (x_min, y_min), (x_max, y_max) = ax.bbox.get_points()
        #x label
        box = np.array(ax.get_xaxis().get_label().get_window_extent(None).get_points())
        bboxes.append(("x_label", [box[0][0], box[0][1], box[1][0], box[1][1]], str(x_label)))

        #y label
        box = np.array(ax.get_yaxis().get_label().get_window_extent(None).get_points())
        bboxes.append(("y_label", [box[0][0], box[0][1], box[1][0], box[1][1]], str(y_label)))

        ## bbox for ticks
        xtick_label_idx = 0
        for label in ax.get_xticklabels():
            bbox = label.get_window_extent().get_points()
        #     if bbox[0][0] < x_min or bbox[1][0] > x_max:
        #         continue
            (x_min_bbox, y_min_bbox), (x_max_bbox, y_max_bbox) = bbox
            label = label.get_text().split('\n')
            height = y_max_bbox - y_min_bbox
            num_lines = len(label)
            sub_height = height/num_lines
            for i, tb in enumerate(label):
                new_bbox = [x_min_bbox, y_min_bbox + sub_height*i, x_max_bbox, y_min_bbox + sub_height*(i+1)]
                bboxes.append((f'x_tick', new_bbox, tb))
                # bboxes.append((f'xtick_{xtick_label_idx}', bbox))
                xtick_label_idx += 1

        ytick_label_idx = 0
        for label in ax.get_yticklabels():
            bbox = label.get_window_extent().get_points()
            # if bbox[0][1] < y_min or bbox[1][1] > y_max:
            #     continue
            (x_min_bbox, y_min_bbox), (x_max_bbox, y_max_bbox) = bbox
            label = label.get_text().split('\n')
            height = y_max_bbox - y_min_bbox
            num_lines = len(label)
            sub_height = height/num_lines
            for i, tb in enumerate(label):
                new_bbox = [x_min_bbox, y_min_bbox + sub_height*i, x_max_bbox, y_min_bbox + sub_height*(i+1)]
                bboxes.append((f'y_tick', new_bbox, tb))
                # bboxes.append((f'xtick_{xtick_label_idx}', bbox))
                ytick_label_idx += 1
                
        # Bar
        for j in range(len(bar_list[0])):
            for i in range(len(bar_list)):
                bbox = bar_list[i][j].get_window_extent().get_points()
                (x_min, y_min), (x_max, y_max) = bbox
                new_bbox = [x_min, y_min, x_max, y_max]
                bboxes.append((f'bar', new_bbox, None))
                
        # Value
        for j in range(len(val_list_for_bbox[0])):
            for i in range(len(val_list_for_bbox)):
                bbox = val_list_for_bbox[i][j].get_window_extent().get_points()
                (x_min, y_min), (x_max, y_max) = bbox
                new_bbox = [x_min, y_min, x_max, y_max]
                bboxes.append((f'val', new_bbox, val_list_for_bbox[i][j].get_text()))
                
        # legend
        for i, patch in enumerate(ax.get_legend().get_patches()):
            bbox = patch.get_window_extent().get_points()
            (x_min, y_min), (x_max, y_max) = bbox
            new_bbox = [x_min, y_min, x_max, y_max]
            bboxes.append((f'v_legend', new_bbox, None))


        for i, label in enumerate(ax.get_legend().get_texts()):
            bbox = label.get_window_extent().get_points()
            (x_min, y_min), (x_max, y_max) = bbox
            new_bbox = [x_min, y_min, x_max, y_max]
            bboxes.append((f't_legend', new_bbox, label.get_text()))

        bbox = fig.get_window_extent(None).transformed(fig.dpi_scale_trans.inverted())
        plt_size = bbox.width*fig.dpi, bbox.height*fig.dpi
        visual_bbox(bboxes, plt_size, 'data/bar/barh_'+str(img_idx)+'.png', 'data/bar_bbox/barh_bbox_'+str(img_idx)+'.png')

        final_elem = {
            'qid':tqa['QAS']['qid'],
            'question':tqa['QAS']['question'],
            'answer':tqa['QAS']['answer'],
            'image':'barh_'+ str(img_idx) + '.png' ,
            'bboxes':bboxes
        }
        img_idx += 1
        final_result.append(final_elem)
        fig.clf()
        plt.close(fig)

# with open('barh_annotation.pkl',"wb") as f:
#     pickle.dump(final_result, f)

# with open('bar_annotation.pkl', 'rb') as handle:
#     print(pickle.load(handle))

In [72]:

# with open('bar_annotation.pkl', 'rb') as handle:
#     print(pickle.load(handle))